In [1]:
import pandas as pd
from tqdm import tqdm 
import numpy as np



In [2]:
vaso = pd.read_csv('feature/vaso.csv')
adm_time_vol_dict = dict()
for i in tqdm(range(len(vaso))):
    # print(vaso.loc[i, 'weightgroup'])
    try:
        if '110' in vaso.loc[i, 'weightgroup']:
            weight = 115
        elif '59-' in vaso.loc[i, 'weightgroup']:
            weight = 55
        else:
            weight = int(vaso.loc[i, 'weightgroup'].split('-')[0]) + 5.0
    except:
        # print(vaso.loc[i, 'weightgroup'])
        continue
    time_d = vaso.loc[i,'time_d']
    adm = vaso.loc[i, 'admissionid']
    start = vaso.loc[i, 'start']
    stop = vaso.loc[i, 'stop']
    dose = vaso.loc[i, 'dose']
    unit = vaso.loc[i, 'doseunit']
    if 'Âµg' in unit:
        pass
    elif unit != "mg":
        print(unit)
    # print(type(start), type(stop), type(dose))
    start_4h = int(start / 60 / 4)
    stop_4h = int(stop/60/4)
    if adm not in adm_time_vol_dict:
        adm_time_vol_dict[adm] = dict()
        for t in range(int(time_d / 60 / 4)):
            adm_time_vol_dict[adm][t] = 0
    for time_4h in range(start_4h, stop_4h + 1):
        overlap_start = max(start, time_4h * 60 * 4)
        overlap_end = min(stop, (time_4h + 1) * 60 * 4)
        if overlap_end > overlap_start:
            if time_4h not in adm_time_vol_dict[adm]:
                adm_time_vol_dict[adm][time_4h] = 0
            adm_time_vol_dict[adm][time_4h] += (overlap_end - overlap_start) / (stop - start) * dose / weight * 1000 / 60 / 4
vs_list = [0, 0, 0, 0, 0]
th = [0, 0.08, 0.22, 0.45]

for k,vs in adm_time_vol_dict.items():
    for t in range(max(vs.keys())):
        if t not in vs:
            adm_time_vol_dict[k][t] = 0
for k,vs in adm_time_vol_dict.items():
    for v in vs.values():
        for i in range(4):
            if v <= th[i]:
                break
            else:
                i += 1
        vs_list[i] += 1
print([vs / sum(vs_list)  for vs in vs_list])


wf = open('feature/vaso_dose.csv', 'w')
wf.write('icustayid,charttime,vaso_dose')
for k,vs in adm_time_vol_dict.items():
    for t in sorted(vs.keys()):
        wf.write('\n{:d},{:d},{:1.4f}'.format(k,t,vs[t]))
        
        
    

100%|██████████| 95867/95867 [00:07<00:00, 13162.50it/s]


[0.8995603697574511, 0.0604115473981579, 0.01658057937582535, 0.009379335707002324, 0.014068167761563279]


In [3]:

vaso = pd.read_csv('feature/fluidin.csv')
adm_time_vol_dict = dict()
for i in tqdm(range(len(vaso))):
    # print(vaso.loc[i, 'weightgroup'])
    try:
        if '110' in vaso.loc[i, 'weightgroup']:
            weight = 115
        else:
            weight = int(vaso.loc[i, 'weightgroup'].split('-')[0]) + 5.0
    except:
        # print(vaso.loc[i, 'weightgroup'])
        continue
    time_d = vaso.loc[i,'time_d']
    adm = vaso.loc[i, 'admissionid']
    start = vaso.loc[i, 'start']
    stop = vaso.loc[i, 'stop']
    dose = vaso.loc[i, 'fluidin']
    # print(type(start), type(stop), type(dose))
    start_4h = int(start / 60 / 4)
    stop_4h = int(stop/60/4)
    if adm not in adm_time_vol_dict:
        adm_time_vol_dict[adm] = dict()
        for t in range(int(time_d / 60 / 4)):
            adm_time_vol_dict[adm][t] = 0
    for time_4h in range(start_4h, stop_4h + 1):
        overlap_start = max(start, time_4h * 60 * 4)
        overlap_end = min(stop, (time_4h + 1) * 60 * 4)
        if overlap_end > overlap_start:
            if time_4h not in adm_time_vol_dict[adm]:
                adm_time_vol_dict[adm][time_4h] = 0
            adm_time_vol_dict[adm][time_4h] += (overlap_end - overlap_start) / (stop - start) * dose  # / weight
vs_list = [0, 0, 0, 0, 0]
th = [0, 50, 180, 530]


for k,vs in adm_time_vol_dict.items():
    for t in range(max(vs.keys())):
        if t not in vs:
            adm_time_vol_dict[k][t] = 0
for k,vs in adm_time_vol_dict.items():
    for v in vs.values():
        for i in range(4):
            if v <= th[i]:
                break
            else:
                i += 1
        vs_list[i] += 1
print([vs / sum(vs_list)  for vs in vs_list])
        
        

wf = open('feature/iv_dose.csv', 'w')
wf.write('icustayid,charttime,iv_dose')
for k,vs in adm_time_vol_dict.items():
    for t in sorted(vs.keys()):
        wf.write('\n{:d},{:d},{:1.4f}'.format(k,t,vs[t]))
            

100%|██████████| 756787/756787 [00:42<00:00, 17620.57it/s]


[0.7777991588821015, 0.018068507712459242, 0.06224511327487887, 0.08779317364744978, 0.05409404648311062]


In [4]:

output = pd.read_csv('sofa/sofa_renal_urine_output.csv')
adm_time_vol_dict = dict()
for i in tqdm(range(len(output))):
    time = output.loc[i,'time']
    adm = output.loc[i, 'admissionid']
    value = output.loc[i, 'value']

    t = int(t / 60 / 4)
    if adm not in adm_time_vol_dict:
        adm_time_vol_dict[adm] = dict()
        if t not in adm_time_vol_dict[adm]:
            adm_time_vol_dict[adm][t] = 0
    adm_time_vol_dict[adm][t] += value

wf = open('feature/output.csv', 'w')
wf.write('icustayid,charttime,output_4h')
for k,vs in adm_time_vol_dict.items():
    for t in sorted(vs.keys()):
        wf.write('\n{:d},{:d},{:1.4f}'.format(k,t,vs[t]))
            

100%|██████████| 471628/471628 [00:10<00:00, 43746.50it/s]
